In [ ]:
#imports
import pandas as pd
import itertools 
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/Shared drives/641 project/consol_reddit_emotion.csv')
#data = pd.read_csv('/content/drive/Shared drives/641 project/reddit_emotion_posterior.csv')
data.head()

,Unnamed: 0,person,anger sl,surprise sl,joy sl,sadness sl,love sl,fear sl,Angry te,Fear te,Happy te,Sad te,Surprise te,Angry temax,Fear temax,Happy temax,Sad temax,Surprise temax
0,0,spoopydoopyjoopy,11,168,52,129,174,243,30.28,163.68,82.90,124.17,142.65,37,117,346,114,163
1,1,BadKarma-18,15,96,24,36,82,105,7.21,94.24,39.02,57.18,63.24,8,90,141,51,68
2,2,Peoerson,8,106,35,39,115,88,17.56,115.34,37.28,55.06,74.54,15,119,135,47,75
3,3,Doomed,18,279,103,110,272,217,37.23,362.97,93.87,190.08,175.23,32,404,247,154,162
4,4,lobester250,1,3,1,8,2,823,1.00,1.00,0.00,3.00,1.00,1,1,832,3,1


In [ ]:
#List of column names: data.columns.tolist()
#BLUE -> Keeping columns: 'Angry te','Fear te','Happy te','Sad te', 'Surprise te'
#YELLOW -> Keeping columns: 'anger sl','surprise sl','joy sl','sadness sl','love sl','fear sl'
#GREEN -> Keeping columns: 'Angry temax','Fear temax','Happy temax','Sad temax','Surprise temax'
#Keep only columns from: green
data = data[['Angry temax','Fear temax','Happy temax','Sad temax','Surprise temax']]
#Creating a new column with the total count of tweets for each user
data['total_tweets'] = data.sum(axis=1)
#Need to rename columns
#blue: 
#data = data.rename(columns={"Angry te": "anger", "Fear te": "fear", "Happy te": "joy", "Sad te": "sadness", "Surprise te": "surprise"})
#yellow: 
#data = data.rename(columns={"anger sl": "anger", "fear sl": "fear", "joy sl": "joy", "sadness sl": "sadness", "surprise sl": "surprise",'love sl':'love'})
#green:
data = data.rename(columns={"Angry temax": "anger", "Fear temax": "fear", "Happy temax": "joy", "Sad temax": "sadness", "Surprise temax": "surprise"})
data.head()

,anger,fear,joy,sadness,surprise,total_tweets
0,37,117,346,114,163,777
1,8,90,141,51,68,358
2,15,119,135,47,75,391
3,32,404,247,154,162,999
4,1,1,832,3,1,838


In [ ]:
#Update values to probabilities
data['anger'] = data['anger']/data['total_tweets']
data['fear'] = data['fear']/data['total_tweets']
data['joy'] = data['joy']/data['total_tweets']
data['sadness'] = data['sadness']/data['total_tweets']
data['surprise'] = data['surprise']/data['total_tweets'] 
#data['love'] = data['love']/data['total_tweets'] #added for yellow
data = data.drop(columns=['total_tweets'])
data.head()

,anger,fear,joy,sadness,surprise
0,0.047619,0.150579,0.445302,0.146718,0.209781
1,0.022346,0.251397,0.393855,0.142458,0.189944
2,0.038363,0.304348,0.345269,0.120205,0.191816
3,0.032032,0.404404,0.247247,0.154154,0.162162
4,0.001193,0.001193,0.992840,0.003580,0.001193


In [ ]:
#Reference: https://medium.com/analytics-vidhya/apriori-algorithm-in-association-rule-learning-9287fe17e944#:~:text=What%20is%20Apriori%20Algorithm%3F,which%20are%20frequently%20visited%20etc.
#Step 1: Create a table which has support count of all the items present in the transaction database
#This is counting in how many transactions an emotion appears*its probability
#so we can just do a sum of all rows for each column
total = pd.DataFrame(data.sum())
total.columns= ['Support count']
total

,Support count
anger,20.188824
fear,108.371870
joy,218.776121
sadness,71.798180
surprise,91.865004


In [ ]:
#Now filter out non-frequent items using minimum support
min_support_count = 1 #For these we don't want to discard any emotions, this step can be skipped for this use case
new_emotions = total[total['Support count'] > min_support_count]
new_emotions

,Support count
anger,20.188824
fear,108.371870
joy,218.776121
sadness,71.798180
surprise,91.865004


In [ ]:
#Create all combinations of emotions
combinations = []
for i in range(2,len(data.columns.tolist())+1): 
    combinations = combinations + list(itertools.combinations(list(new_emotions.index), i))
combinations

[('anger', 'fear'),
 ('anger', 'joy'),
 ('anger', 'sadness'),
 ('anger', 'surprise'),
 ('fear', 'joy'),
 ('fear', 'sadness'),
 ('fear', 'surprise'),
 ('joy', 'sadness'),
 ('joy', 'surprise'),
 ('sadness', 'surprise'),
 ('anger', 'fear', 'joy'),
 ('anger', 'fear', 'sadness'),
 ('anger', 'fear', 'surprise'),
 ('anger', 'joy', 'sadness'),
 ('anger', 'joy', 'surprise'),
 ('anger', 'sadness', 'surprise'),
 ('fear', 'joy', 'sadness'),
 ('fear', 'joy', 'surprise'),
 ('fear', 'sadness', 'surprise'),
 ('joy', 'sadness', 'surprise'),
 ('anger', 'fear', 'joy', 'sadness'),
 ('anger', 'fear', 'joy', 'surprise'),
 ('anger', 'fear', 'sadness', 'surprise'),
 ('anger', 'joy', 'sadness', 'surprise'),
 ('fear', 'joy', 'sadness', 'surprise'),
 ('anger', 'fear', 'joy', 'sadness', 'surprise')]

In [ ]:
DFCombinations = pd.DataFrame(combinations)
DFCombinations.columns = ['Emotion1','Emotion2','Emotion3','Emotion4','Emotion5']#,'Emotion6'] #update based on number of emotions
DFCombinations.fillna('',inplace=True)
#update based on number of emotions:
DFCombinations['Combination'] = DFCombinations['Emotion1']+' '+DFCombinations['Emotion2']+' '+DFCombinations['Emotion3']+' '+DFCombinations['Emotion4']+' '+DFCombinations['Emotion5']#+' '+DFCombinations['Emotion6']
DFCombinations['Combination'] = DFCombinations['Combination'].str.strip()
DFCombinations

,Emotion1,Emotion2,Emotion3,Emotion4,Emotion5,Combination
0,anger,fear,,,,anger fear
1,anger,joy,,,,anger joy
2,anger,sadness,,,,anger sadness
3,anger,surprise,,,,anger surprise
4,fear,joy,,,,fear joy
5,fear,sadness,,,,fear sadness
6,fear,surprise,,,,fear surprise
7,joy,sadness,,,,joy sadness
8,joy,surprise,,,,joy surprise
9,sadness,surprise,,,,sadness surprise


In [ ]:
#Count support for the combinations 
data_w_combinations = data
for i in range(0,len(DFCombinations['Combination'])):
    data_w_combinations[DFCombinations['Combination'][i]] = 1
    for j in ['Emotion1','Emotion2','Emotion3','Emotion4','Emotion5']:#,'Emotion6']: #update based on number of emotions
        if (DFCombinations[j][i] != ''):
            data_w_combinations[DFCombinations['Combination'][i]] = data_w_combinations[DFCombinations['Combination'][i]] * data_w_combinations[DFCombinations[j][i]] 
data_w_combinations

,anger,fear,joy,sadness,surprise,anger fear,anger joy,anger sadness,anger surprise,fear joy,...,fear joy sadness,fear joy surprise,fear sadness surprise,joy sadness surprise,anger fear joy sadness,anger fear joy surprise,anger fear sadness surprise,anger joy sadness surprise,fear joy sadness surprise,anger fear joy sadness surprise
0,0.047619,0.150579,0.445302,0.146718,0.209781,0.007170,0.021205,0.006987,0.009990,0.067053,...,0.009838,0.014067,4.634632e-03,0.013706,4.684729e-04,6.698340e-04,2.206968e-04,6.526588e-04,2.063813e-03,9.827681e-05
1,0.022346,0.251397,0.393855,0.142458,0.189944,0.005618,0.008801,0.003183,0.004245,0.099014,...,0.014105,0.018807,6.802562e-03,0.010657,3.152025e-04,4.202700e-04,1.520126e-04,2.381530e-04,2.679221e-03,5.987087e-05
2,0.038363,0.304348,0.345269,0.120205,0.191816,0.011676,0.013246,0.004611,0.007359,0.105082,...,0.012631,0.020156,7.017393e-03,0.007961,4.845770e-04,7.732612e-04,2.692095e-04,3.054057e-04,2.422885e-03,9.294956e-05
3,0.032032,0.404404,0.247247,0.154154,0.162162,0.012954,0.007920,0.004938,0.005194,0.099988,...,0.015414,0.016214,1.010929e-02,0.006181,4.937272e-04,5.193754e-04,3.238211e-04,1.979797e-04,2.499494e-03,8.006387e-05
4,0.001193,0.001193,0.992840,0.003580,0.001193,0.000001,0.001185,0.000004,0.000001,0.001185,...,0.000004,0.000001,5.097875e-09,0.000004,5.061375e-09,1.687125e-09,6.083383e-12,5.061375e-09,5.061375e-09,6.039827e-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,0.000000,0.108108,0.729730,0.108108,0.054054,0.000000,0.000000,0.000000,0.000000,0.078890,...,0.008529,0.004264,6.317494e-04,0.004264,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.610063e-04,0.000000e+00
507,0.010309,0.123711,0.525773,0.082474,0.257732,0.001275,0.005420,0.000850,0.002657,0.065044,...,0.005364,0.016764,2.629638e-03,0.011176,5.530374e-05,1.728242e-04,2.710967e-05,1.152161e-04,1.382593e-03,1.425354e-05
508,0.100000,0.600000,0.200000,0.000000,0.100000,0.060000,0.020000,0.000000,0.010000,0.120000,...,0.000000,0.012000,0.000000e+00,0.000000,0.000000e+00,1.200000e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
509,0.025961,0.178609,0.521288,0.128764,0.145379,0.004637,0.013533,0.003343,0.003774,0.093106,...,0.011989,0.013536,3.343484e-03,0.009758,3.112352e-04,3.513946e-04,8.679866e-05,2.533310e-04,1.742917e-03,4.524707e-05


In [ ]:
new_total = pd.DataFrame(data_w_combinations.sum())
new_total.columns= ['Support count']
new_total['Number of emotions'] = new_total.index.str.count(' ')+1
new_total

,Support count,Number of emotions
anger,20.188824,1
fear,108.371870,1
joy,218.776121,1
sadness,71.798180,1
surprise,91.865004,1
anger fear,4.267327,2
anger joy,7.048362,2
anger sadness,2.467862,2
anger surprise,3.532097,2
fear joy,36.267199,2


In [ ]:
#Now filter out non-frequent items using minimum support
#since we are multiplying probabilities the numbers are very small and we can't use the same min_support in each step
means = new_total[new_total['Number of emotions'] == 2]['Support count'].mean()
min_support_count = [0,0] + new_total.groupby('Number of emotions').mean()['Support count'].tolist()[1:] #Ignore index 0 and 1 for 0 and 1 emotions
new_combinations = pd.DataFrame()
for i in range(0,len(new_total)):
  if new_total['Support count'][i] >= min_support_count[new_total['Number of emotions'][i]]:
    new_combinations = new_combinations.append(new_total.iloc[i]) # 0]#min_support_count[new_total['Number of emotions']]]
new_combinations

,Support count,Number of emotions
anger,20.188824,1.0
fear,108.371870,1.0
joy,218.776121,1.0
sadness,71.798180,1.0
surprise,91.865004,1.0
fear joy,36.267199,2.0
fear surprise,19.407202,2.0
joy sadness,25.978185,2.0
joy surprise,35.043757,2.0
fear joy sadness,4.934727,3.0


In [ ]:
#Only keep combinations of 2 and 3 emotions
final_combinations = new_combinations[(new_combinations['Number of emotions']<4)&(new_combinations['Number of emotions']>1)]
final_combinations

,Support count,Number of emotions
fear joy,36.267199,2.0
fear surprise,19.407202,2.0
joy sadness,25.978185,2.0
joy surprise,35.043757,2.0
fear joy sadness,4.934727,3.0
fear joy surprise,6.573416,3.0
fear sadness surprise,2.818401,3.0
joy sadness surprise,4.527045,3.0


In [ ]:
DFCombinations = DFCombinations.set_index('Combination')
DFCombinations

,Emotion1,Emotion2,Emotion3,Emotion4,Emotion5
Combination,,,,,
anger fear,anger,fear,,,
anger joy,anger,joy,,,
anger sadness,anger,sadness,,,
anger surprise,anger,surprise,,,
fear joy,fear,joy,,,
fear sadness,fear,sadness,,,
fear surprise,fear,surprise,,,
joy sadness,joy,sadness,,,
joy surprise,joy,surprise,,,


In [ ]:
full_table = final_combinations.join(DFCombinations,how='left')
full_table

,Support count,Number of emotions,Emotion1,Emotion2,Emotion3,Emotion4,Emotion5
fear joy,36.267199,2.0,fear,joy,,,
fear surprise,19.407202,2.0,fear,surprise,,,
joy sadness,25.978185,2.0,joy,sadness,,,
joy surprise,35.043757,2.0,joy,surprise,,,
fear joy sadness,4.934727,3.0,fear,joy,sadness,,
fear joy surprise,6.573416,3.0,fear,joy,surprise,,
fear sadness surprise,2.818401,3.0,fear,sadness,surprise,,
joy sadness surprise,4.527045,3.0,joy,sadness,surprise,,


In [ ]:
new_total

,Support count,Number of emotions
anger,20.188824,1
fear,108.371870,1
joy,218.776121,1
sadness,71.798180,1
surprise,91.865004,1
anger fear,4.267327,2
anger joy,7.048362,2
anger sadness,2.467862,2
anger surprise,3.532097,2
fear joy,36.267199,2


In [ ]:
#Step4: Generate association rules for the combinations and calculate confidence: Confidence( I1 -> I2 ) = SupportCount ( I1 U I2 ) / SupportCount( I1 )
full_table['Confidence E1->E2'] = ''
full_table['Confidence E2->E1'] = ''
full_table['Confidence E1,E2->E3'] = ''
full_table['Confidence E1,E3->E2'] = ''
full_table['Confidence E2,E3->E1'] = ''
for i in range(0,len(full_table)):
  full_table['Confidence E1->E2'][i] = (full_table['Support count'][i]/new_total['Support count'][full_table['Emotion1'][i]])*100
  full_table['Confidence E2->E1'][i] = (full_table['Support count'][i]/new_total['Support count'][full_table['Emotion2'][i]])*100
  if (full_table['Number of emotions'][i]==3.0):
    full_table['Confidence E1,E2->E3'][i] = (full_table['Support count'][i]/new_total['Support count'][full_table['Emotion1'][i]+' '+full_table['Emotion2'][i]])*100
    full_table['Confidence E1,E3->E2'][i] = (full_table['Support count'][i]/new_total['Support count'][full_table['Emotion1'][i]+' '+full_table['Emotion3'][i]])*100
    full_table['Confidence E2,E3->E1'][i] = (full_table['Support count'][i]/new_total['Support count'][full_table['Emotion2'][i]+' '+full_table['Emotion3'][i]])*100
full_table

,Support count,Number of emotions,Emotion1,Emotion2,Emotion3,Emotion4,Emotion5,Confidence E1->E2,Confidence E2->E1,"Confidence E1,E2->E3","Confidence E1,E3->E2","Confidence E2,E3->E1"
fear joy,36.267199,2.0,fear,joy,,,,33.46551,16.577311,,,
fear surprise,19.407202,2.0,fear,surprise,,,,17.90797,21.125783,,,
joy sadness,25.978185,2.0,joy,sadness,,,,11.874324,36.182234,,,
joy surprise,35.043757,2.0,joy,surprise,,,,16.018091,38.147015,,,
fear joy sadness,4.934727,3.0,fear,joy,sadness,,,4.553513,2.255606,13.606584,32.034274,18.995657
fear joy surprise,6.573416,3.0,fear,joy,surprise,,,6.065611,3.004631,18.124962,33.871014,18.757738
fear sadness surprise,2.818401,3.0,fear,sadness,surprise,,,2.600676,3.925449,18.295933,14.522449,21.966783
joy sadness surprise,4.527045,3.0,joy,sadness,surprise,,,2.069259,6.305236,17.426333,12.918264,35.284055


In [ ]:
#Step5: calculate lift for the association rules: Lift ( I1 -> I2 ) = Confidence( I1 -> I2 )/ Support( I2 ) 
full_table['Lift E1->E2'] = ''
full_table['Lift E2->E1'] = ''
full_table['Lift E1,E2->E3'] = ''
full_table['Lift E1,E3->E2'] = ''
full_table['Lift E2,E3->E1'] = ''
for i in range(0,len(full_table)):
  full_table['Lift E1->E2'][i] = (full_table['Confidence E1->E2'][i]/new_total['Support count'][full_table['Emotion2'][i]])*100
  full_table['Lift E2->E1'][i] = (full_table['Confidence E2->E1'][i]/new_total['Support count'][full_table['Emotion1'][i]])*100
  if (full_table['Number of emotions'][i]==3.0):
    full_table['Lift E1,E2->E3'][i] = (full_table['Confidence E1,E2->E3'][i]/new_total['Support count'][full_table['Emotion3'][i]])*100
    full_table['Lift E1,E3->E2'][i] = (full_table['Confidence E1,E3->E2'][i]/new_total['Support count'][full_table['Emotion2'][i]])*100
    full_table['Lift E2,E3->E1'][i] = (full_table['Confidence E2,E3->E1'][i]/new_total['Support count'][full_table['Emotion1'][i]])*100
full_table

,Support count,Number of emotions,Emotion1,Emotion2,Emotion3,Emotion4,Emotion5,Confidence E1->E2,Confidence E2->E1,"Confidence E1,E2->E3","Confidence E1,E3->E2","Confidence E2,E3->E1",Lift E1->E2,Lift E2->E1,"Lift E1,E2->E3","Lift E1,E3->E2","Lift E2,E3->E1"
fear joy,36.267199,2.0,fear,joy,,,,33.46551,16.577311,,,,15.296692,15.296692,,,
fear surprise,19.407202,2.0,fear,surprise,,,,17.90797,21.125783,,,,19.493789,19.493789,,,
joy sadness,25.978185,2.0,joy,sadness,,,,11.874324,36.182234,,,,16.538475,16.538475,,,
joy surprise,35.043757,2.0,joy,surprise,,,,16.018091,38.147015,,,,17.436553,17.436553,,,
fear joy sadness,4.934727,3.0,fear,joy,sadness,,,4.553513,2.255606,13.606584,32.034274,18.995657,2.081357,2.081357,18.951155,14.642491,17.528217
fear joy surprise,6.573416,3.0,fear,joy,surprise,,,6.065611,3.004631,18.124962,33.871014,18.757738,2.77252,2.77252,19.729997,15.482043,17.308678
fear sadness surprise,2.818401,3.0,fear,sadness,surprise,,,2.600676,3.925449,18.295933,14.522449,21.966783,3.622203,3.622203,19.916107,20.226765,20.26982
joy sadness surprise,4.527045,3.0,joy,sadness,surprise,,,2.069259,6.305236,17.426333,12.918264,35.284055,2.88205,2.88205,18.969501,17.992467,16.127928


In [ ]:
#Processing for printing results
#full_table.columns.tolist()
full_table['Confidence E1->E2'] = pd.to_numeric(full_table['Confidence E1->E2'])
full_table['Confidence E2->E1'] = pd.to_numeric(full_table['Confidence E2->E1'])
full_table['Confidence E1,E2->E3'] = pd.to_numeric(full_table['Confidence E1,E2->E3'])
full_table['Confidence E1,E3->E2'] = pd.to_numeric(full_table['Confidence E1,E3->E2'])
full_table['Confidence E2,E3->E1'] = pd.to_numeric(full_table['Confidence E2,E3->E1'])
full_table['Max_Conf'] = full_table[['Confidence E1->E2','Confidence E2->E1','Confidence E1,E2->E3','Confidence E1,E3->E2','Confidence E2,E3->E1']].max(axis=1)
full_table['Rule_Max_Conf'] = full_table[['Confidence E1->E2','Confidence E2->E1','Confidence E1,E2->E3','Confidence E1,E3->E2','Confidence E2,E3->E1']].idxmax(axis=1)
full_table['Rule_Max_Conf'] = full_table['Rule_Max_Conf'].astype(str)
full_table['Rule_Max_Conf'] = full_table['Rule_Max_Conf'].str.replace('Confidence ','')
full_table['Lift_inter'] = 'Lift ' + full_table['Rule_Max_Conf']
full_table['Lift_final'] = full_table.apply(lambda x: x[x['Lift_inter']],axis=1)
full_table = full_table.drop(columns=['Lift_inter'])
full_table["Rule_Max_Conf"]= full_table.apply(lambda x: x['Rule_Max_Conf'].replace('E1', str(x['Emotion1'])), axis=1)
full_table["Rule_Max_Conf"]= full_table.apply(lambda x: x['Rule_Max_Conf'].replace('E2', str(x['Emotion2'])), axis=1)
full_table["Rule_Max_Conf"]= full_table.apply(lambda x: x['Rule_Max_Conf'].replace('E3', str(x['Emotion3'])), axis=1)
full_table

,Support count,Number of emotions,Emotion1,Emotion2,Emotion3,Emotion4,Emotion5,Confidence E1->E2,Confidence E2->E1,"Confidence E1,E2->E3","Confidence E1,E3->E2","Confidence E2,E3->E1",Lift E1->E2,Lift E2->E1,"Lift E1,E2->E3","Lift E1,E3->E2","Lift E2,E3->E1",Max_Conf,Rule_Max_Conf,Lift_final
fear joy,36.267199,2.0,fear,joy,,,,33.465510,16.577311,NaN,NaN,NaN,15.296692,15.296692,,,,33.465510,fear->joy,15.296692
fear surprise,19.407202,2.0,fear,surprise,,,,17.907970,21.125783,NaN,NaN,NaN,19.493789,19.493789,,,,21.125783,surprise->fear,19.493789
joy sadness,25.978185,2.0,joy,sadness,,,,11.874324,36.182234,NaN,NaN,NaN,16.538475,16.538475,,,,36.182234,sadness->joy,16.538475
joy surprise,35.043757,2.0,joy,surprise,,,,16.018091,38.147015,NaN,NaN,NaN,17.436553,17.436553,,,,38.147015,surprise->joy,17.436553
fear joy sadness,4.934727,3.0,fear,joy,sadness,,,4.553513,2.255606,13.606584,32.034274,18.995657,2.081357,2.081357,18.951155,14.642491,17.528217,32.034274,"fear,sadness->joy",14.642491
fear joy surprise,6.573416,3.0,fear,joy,surprise,,,6.065611,3.004631,18.124962,33.871014,18.757738,2.77252,2.77252,19.729997,15.482043,17.308678,33.871014,"fear,surprise->joy",15.482043
fear sadness surprise,2.818401,3.0,fear,sadness,surprise,,,2.600676,3.925449,18.295933,14.522449,21.966783,3.622203,3.622203,19.916107,20.226765,20.26982,21.966783,"sadness,surprise->fear",20.269820
joy sadness surprise,4.527045,3.0,joy,sadness,surprise,,,2.069259,6.305236,17.426333,12.918264,35.284055,2.88205,2.88205,18.969501,17.992467,16.127928,35.284055,"sadness,surprise->joy",16.127928


In [ ]:
#Results for green
#Confidence: Keep rule with highest confidence for each row (20-30% confidence)
#Lift: order by lift (16-19% lift)
results = full_table[['Rule_Max_Conf','Max_Conf','Lift_final']]
results = results.sort_values(by=['Lift_final'],ascending=False)
results

,Rule_Max_Conf,Max_Conf,Lift_final
fear sadness surprise,"sadness,surprise->fear",21.966783,20.269820
fear surprise,surprise->fear,21.125783,19.493789
joy surprise,surprise->joy,38.147015,17.436553
joy sadness,sadness->joy,36.182234,16.538475
joy sadness surprise,"sadness,surprise->joy",35.284055,16.127928
fear joy surprise,"fear,surprise->joy",33.871014,15.482043
fear joy,fear->joy,33.465510,15.296692
fear joy sadness,"fear,sadness->joy",32.034274,14.642491
